In [50]:
import os
import itertools

from iScore.graphrank.kernel import Kernel
import numpy as np
from scipy.sparse import coo_matrix

In [7]:
ker = Kernel(testIDs=None,
             test_graph="test/graph/",
             trainIDs=None,
             train_graph="./graph/",
             train_archive="../iScore/model/training_set.tar.gz",
             method="vect")
ker.import_from_mat()

# get the path of the check file
checkfile = ker.get_check_file(None)

In [109]:
lamb = 1
walk = 3

In [110]:
name1, G1 = list(ker.test_graphs.items())[0]
name2, G2 = list(ker.train_graphs.items())[0]
n1 = os.path.splitext(name1)[0]
n2 = os.path.splitext(name2)[0]

In [111]:
name1, name2, G1, G2

('1ACB_4w.pckl',
 '1E96_383w.pckl',
 <iScore.graphrank.graph.Graph at 0x7fa6802e1430>)

In [112]:
def compute_kron_mat(g1, g2, method="vect", manual_transpose=True):
    """Kroenecker matrix calculation

    Notes:
        Implementation of equation 5 (l=1) in the reference
        (https://doi.org/10.1093/bioinformatics/btz496)

    Args:
        g1 (iScore.Graph): first graph
        g2 (iScore.Graph): second graph

    """
    # double the edges index for g1
    index1 = np.vstack((g1.edges_index, np.flip(g1.edges_index, axis=1)))
    index2 = g2.edges_index

    # double the pssm edges for g1
    pssm1 = np.vstack((g1.edges_pssm, np.hstack((g1.edges_pssm[:, 20:], g1.edges_pssm[:, :20]))))
    pssm2 = g2.edges_pssm

    # compute the weight
    # Note that the weight here is calculating knode(vi, v'i) * knode(vj, v'j)
    # of eq5, and kedge(eI, e'J) of eq5 is set to 1.
    if method == 'iter':
        # the trick here is that _rbf_kernel is actually calculating
        # knode(vi, v'i) * knode(vj, v'j) due to the shape of e.g. p[0]
        # is (40,) but not (20,).
        weight = np.array([_rbf_kernel(p[0], p[1]) for p in itertools.product(*[pssm1, pssm2])])
        ind = np.array([_get_index(k[0], k[1], g2.num_nodes) for k in itertools.product(*[index1, index2])])
    elif method == 'combvec':
        weight = _rbf_kernel_combvec(pssm1, pssm2)
        ind = _get_index_combvec(index1, index2, g2.num_nodes)
    elif method == 'vect':
        weight = _rbf_kernel_vectorized(pssm1, pssm2)
        ind = _get_index_vect(index1, index2, g2.num_nodes)
    else:
        raise ValueError('Method %s not recognized' % method)

    # final size
    n_nodes_prod = g1.num_nodes * g2.num_nodes

    # instead of taking the transpose we duplicate
    # the weight and indexes (with switch)
    if manual_transpose:
        weight = np.concatenate((weight, weight))
        ind = np.vstack((ind, np.flip(ind, axis=1)))
        index = (ind[:,0], ind[:,1])
        # create the matrix
        Wx = coo_matrix((weight, index), shape=(n_nodes_prod, n_nodes_prod))
    else:
        index = (ind[:,0], ind[:,1])
        Wx = coo_matrix((weight, index), shape=(n_nodes_prod, n_nodes_prod))
        Wx += Wx.transpose()

    return Wx

In [113]:
def compute_W0(g1, g2, method='vect'):
    """Calculation of t W0 vector from the nodes pssm similarity

    Notes:
        Implementation of equation 5 (l=0) in the reference
        (https://doi.org/10.1093/bioinformatics/btz496)

    Args:
        g1 (iScore.Graph): first graph
        g2 (iScore.Graph): second graph
        method (str, optional): options: iter, combvec, vect (default vect)

    """
    if method == 'iter':
        W0 = np.array([_rbf_kernel(p[0], p[1]) for p in itertools.product(*[g1.nodes_pssm_data, g2.nodes_pssm_data])])
    elif method == 'combvec':
        W0 = _rbf_kernel_combvec(g1.nodes_pssm_data, g2.nodes_pssm_data)
    elif method == 'vect':
        W0 = _rbf_kernel_vectorized(g1.nodes_pssm_data, g2.nodes_pssm_data)
    else:
        raise ValueError('Method %s not recognized' % method)
    return W0

In [114]:
def compute_px(g1, g2, cutoff=0.5):
    """Calculation of the Px vector from the nodes info.

    Notes:
        Implementation of equations 7 and 8 in the reference
        (https://doi.org/10.1093/bioinformatics/btz496)

    Args:
        g1 (iScore.Graph): first graph
        g2 (iScore.Graph): second graph
        cutoff (float, optional): if px[i]<cuoff -> px[i]=0
    """
    px = [t[0]*t[1] if (float(t[0])>cutoff or float(t[1])>cutoff) else 0 for t in itertools.product(*[g1.nodes_info_data, g2.nodes_info_data])]
    return px

In [115]:
def _rbf_kernel_vectorized(data1, data2, sigma=10):
    """kernel for node similarity computed with the vectorized method

    Notes:
        Implementation of equation 6 in the reference
        (https://doi.org/10.1093/bioinformatics/btz496)

    Args:
        data1 (np.array): pssm data 1
        data2 (np.array): pssm data 2
        sigma (int, optional): exponent of the exponetial

    Returns:
        np.array: value of the rbk kernel for all the pairs
    """
    delta = -2 * np.dot(data1, data2.T) + np.sum(data1 ** 2, axis=1)[:, None] + np.sum(data2 ** 2, axis=1)
    beta = 2 * sigma ** 2
    return np.exp(-delta / beta).reshape(-1)


def _rbf_kernel(data1, data2, sigma=10):
    """Kernel for the node similarity calculation using PSSM data.
    Used in the iter method.

    Notes:
        Implementation of equation 6 in the reference
        (https://doi.org/10.1093/bioinformatics/btz496)

    Args:
        data1 (np.array): pssm data 1
        data2 (np.array): pssm data 2
        sigma (int, optional): exponent of the exponetial

    Returns:
        float: value of the rbk kernel
    """
    delta = np.sum((data1 - data2) ** 2)
    beta = 2 * sigma **2
    return np.exp(-delta / beta)


def _combvec(a1, a2, axis=0):
    """Returns all the combination of the column vectors contained in a1 and a2.

    Args:
        a1 (np.array): matrix of vectors
        a2 (np.array): matrix of vectors
        axis (int, optional): axis for the combination

    Returns:
        np.array: matrix holding the all the combination of the vectors
    """
    n1, m1 = a1.shape
    n2, m2 = a2.shape
    if axis == 0:
        return np.vstack((np.repeat(a1, m2, axis=1), np.tile(a2, (1, m1))))
    if axis == 1:
        return np.hstack((np.repeat(a1, n2, axis=0), np.tile(a2, (n1, 1))))

    
def _rbf_kernel_combvec(data1, data2, sigma=10):
    """kernel for node similarity computed with the combvec method

    Notes:
        Implementation of equation 6 in the reference
        (https://doi.org/10.1093/bioinformatics/btz496)

    Args:
        data1 (np.array): pssm data 1
        data2 (np.array): pssm data 2
        sigma (int, optional): exponent of the exponetial

    Returns:
        np.array: value of the rbk kernel for all the pairs
    """
    k = data1.shape[1]
    data = _combvec(data1, data2, axis=1)
    data = np.sum((data[:, :k] - data[:, k:]) ** 2, 1)
    beta = 2 * sigma **2
    return np.exp(-data / beta)

In [116]:
def _get_index(index1, index2, size2):
    """Get the index in the bigraph iter method

    Args:
        index1 (list(int)): List of the edge index in the first graph
        index1 (list(int)): List of the edge index in the second graph
        size2 (int): Number of nodes in the second graph

    Returns:
        list(int): List of index in the bigraph
    """
    index = np.array(index1.tolist()) * size2 + np.array(index2.tolist())
    return index.tolist()


def _get_index_combvec(index1, index2, size2):
    """Get the index in the bigraph combvec method

    Args:
        index1 (list(int)): List of the edge index in the first graph
        index1 (list(int)): List of the edge index in the second graph
        size2 (int): Number of nodes in the second graph

    Returns:
        list(int): List of index in the bigraph
    """
    index = _combvec(index1, index2, axis=1)
    return index[:, :2]*float(size2) + index[:, 2:]


def _get_index_vect(index1, index2, size2):
    """Get the index in the bigraph vect method

    Args:
        index1 (list(int)): List of the edge index in the first graph
        index1 (list(int)): List of the edge index in the second graph
        size2 (int): Number of nodes in the second graph

    Returns:
        list(int): List of index in the bigraph
    """
    index1 = index1*float(size2)
    return np.hstack((
        (index1[:, 0][:, np.newaxis] + index2[:, 0]).reshape(-1, 1), 
        (index1[:, 1][:, np.newaxis] + index2[:, 1]).reshape(-1,1)))

In [117]:
def compute_K(px, W0, Wx, lamb=1, walk=3):
    """Compute random walk graph kernel

    Notes:
        Implementation of equation 4 in the reference
        (https://doi.org/10.1093/bioinformatics/btz496)

    Args:
        lamb (int, optional): value of lambda
        walk (int, optional): length of the walk

    Returns:
        list(float): values of the kernel
    """
    px /= np.sum(px)
    K = np.zeros(walk + 1)
    K[0] = np.sum(px ** 2 * W0)
    pW = Wx.dot(px)
    for i in range(1, walk+1):
        K[i] = K[i-1] + lamb**i * np.sum(pW * px)
        pW = Wx.dot(pW)
    return K

In [118]:
W0 = compute_W0(G1, G2)
Wx = compute_kron_mat(G1, G2)
px = compute_px(G1, G2)

K[(n1, n2)] = compute_K(px, W0, Wx, lamb=lamb, walk=walk)

In [120]:
# We have to go through this process every time we want to score a new test set 
# which will be quite painful because if there are N training examples and M test examples
# it will have to always create the N x M matrix before doing any scoring.